
<div hidden>

$\gdef\dd{\mathrm{d}}$

</div>

<div hidden>

$\gdef\abs#1{\left\vert#1\right\vert}$

</div>

<div hidden>

$\gdef\ve#1{\bm{#1}}$
</div>

<div hidden>

$\gdef\mat#1{\mathbf{#1}}$
</div>


# Sampling with CUQIpy

In this notebook, we explore uncertainty quantification for inverse problems through Bayesian inference using CUQIpy. We focus on exploring the sampling capabilities of CUQIpy, for two target distribution: 
- a bi-variate "donut" distribution,
- and a posterior distribution for a 1D Poisson-based BIP.

The former is used for illustrative purposes and is not associated with an inverse problem, while the latter is a more realistic example of a BIP.

## <font color=#CD853F> Contents of this notebook: </font>

## <font color=#CD853F> Learning objectives: </font> <a name="r-learning-objectives"></a>


The section marked with ★ is optional and can be skipped if you are short on time.

## References
[1] Gelman, Andrew, et al. "Bayesian workflow." arXiv preprint arXiv:2011.01808 (2020) https://arxiv.org/abs/2011.01808.

[2] Riis, Nicolai AB, et al. "CUQIpy--Part I: computational uncertainty quantification for inverse problems in Python." arXiv preprint arXiv:2305.16949 (2023) https://arxiv.org/abs/2305.16949.


In [ ]:
from cuqi.distribution import DistributionGallery
import numpy as np
import matplotlib.pyplot as plt
from cuqi.sampler import MH, CWMH
import time
import scipy.stats as sps

In [ ]:

def plot2d(val, x1_min, x1_max, x2_min, x2_max, N2=201):
    # plot
    pixelwidth_x = (x1_max-x1_min)/(N2-1)
    pixelwidth_y = (x2_max-x2_min)/(N2-1)

    hp_x = 0.5*pixelwidth_x
    hp_y = 0.5*pixelwidth_y

    extent = (x1_min-hp_x, x1_max+hp_x, x2_min-hp_y, x2_max+hp_y)

    plt.imshow(val, origin='lower', extent=extent)
    plt.colorbar()


def plot_pdf_2D(distb, x1_min, x1_max, x2_min, x2_max, N2=201):
    N2 = 201
    ls1 = np.linspace(x1_min, x1_max, N2)
    ls2 = np.linspace(x2_min, x2_max, N2)
    grid1, grid2 = np.meshgrid(ls1, ls2)
    distb_pdf = np.zeros((N2,N2))
    for ii in range(N2):
        for jj in range(N2):
            distb_pdf[ii,jj] = np.exp(distb.logd(np.array([grid1[ii,jj], grid2[ii,jj]]))) 
    plot2d(distb_pdf, x1_min, x1_max, x2_min, x2_max, N2)

## <font color=#CD853F> The "dount" distribution </font> <a name="r-donut"></a>

In CUQIpy, we provide a set of bi-variate distributions for illustrative purposes. One of these is the "donut" distribution, which is a bi-variate distribution of a donut-shaped. The distribution is defined as follows:

$$

\begin{aligned}
log (p(\mathbf{x})) \propto - \frac{1}{\sigma_\text{donut}^2} \left( \left\| \mathbf{x} \right\| - r_\text{donut} \right)^2

\end{aligned}

$$

Where $\mathbf{x} = (x_1, x_2)$ is a 2D vector, $\left\| \mathbf{x} \right\|$ is the Euclidean norm of $\mathbf{x}$, $r_\text{donut}$ is the radius of the donut, and $\sigma_\text{donut}$ is a scalar value that controls the width of the "donut".

To load the "donut" distribution, we use the following:

In [ ]:

target_donut = DistributionGallery("donut")

print(target_donut)

We can plot the distribution probability density function (pdf):

In [ ]:
plot_pdf_2D(target_donut, -4, 4, -4, 4)


## <font color=#CD853F> A 1D Poisson-based BIP </font> <a name="r-donut"></a>

##### <font color=#8B4513> The forward model </font> <a name="r-forward-model"></a>

Consider a heat conductive rod of length $L = \pi$ with a varying conductivity (the conductivity of the rod changes from point to point). We fix the temperature at the end-points of the rod and apply a heat source distributed along the length of the rod. We wait until the rod reaches an equilibrium temperature distribution. The equilibrium temperature of the rod is modelled using the second order steady-state PDE as

$$
\left\{
\begin{aligned}
& \dfrac{\dd}{\dd \xi}\left(u(\xi) \dfrac{\dd y(\xi)}{\dd \xi}\right) = -f(\xi), \quad & \xi\in (0,L) \\
& y(0) = y(L) = 0.
\end{aligned}
\right.
$$
Here, $y$ represents the temperature distribution along the rod, $u(\xi) $ is the unknown conductivity of the rod and $f(\xi)$ is a deterministic heat source given by

$$
\begin{aligned}
	f(\xi) = 10\exp( -\frac{ (\xi - L/2)^2} {0.02} ).
\end{aligned}
$$

To ensure that the conductivity of the rod is non-negative, we parameterize $u$ by a random variable $x$ as follows:
 
$$
 \begin{aligned}
 u( \cdot  ) = \exp( x( \cdot  ) )
 \end{aligned}
$$
where $x$ is not necessarily positive.

Let us load the forward model that maps the random variable $x$ to the temperature distribution $y$ in CUQIpy. We will use the following parameters:
* `dim` : number of discretization points for the rod
* `L` : length of the rod
* `f` : a function that represents the heat source

In [ ]:
dim = 201
L = np.pi

xs = np.array([0.2, 0.4, 0.6, 0.8])*L
ws = 0.8
sigma_s = 0.05
def source(t):
    s = np.zeros(N-1)
    for i in range(4):
        s += ws * sps.norm.pdf(t, loc=xs[i], scale=sigma_s)
    return s


Then we can load the 1D Poisson forward model as follows:

In [ ]:
A, _, _ = cuqi.testproblem.Poisson1D(dim=dim, 
                                    endpoint=L,
                                    field_type='KL',
                                    field_params={'num_modes': 10} ,
                                    map=lambda x: np.exp(x), 
                                    source=source).get_components()

We print the forward model to see its details.

In [ ]:
A

We can look at the domain and range geometries of the forward model.

In [ ]:
print(A.domain_geometry)
print(A.range_geometry)

These geometries are of type `Continuous1D` which represents a 1D continuous signal/field defined on a grid. We can view the grid:

In [ ]:
print(A.domain_geometry.grid)

Additionally, the properties `domain_dim` and `range_dim` of the forward model represent the dimension of the input and output of the forward model, respectively.

In [ ]:
print(A.domain_dim)
print(A.range_dim)

Let us create an array representing a constant conductivity over the grid nodes 

In [ ]:
some_x_array = 20*np.ones(A.domain_dim)

We can wrap the array in a `CUQIarray` object which is the main data structure in CUQIpy for variables (e.g. arrays and fields)

In [ ]:
some_x = CUQIarray(some_x_array, geometry=A.domain_geometry)

Note that we pass `geometry=A.domain_geometry` to equip the `CUQIarray` object with the same geometry as the domain of the forward model, which is has information about what the values of the array represent (e.g. function values on a grid for this case).

We can plot the conductivity array using the `plot` method of the `CUQIarray` object

In [ ]:
some_x.plot()

We can also evaluate the forward model at the conductivity array and plot the solution:

In [ ]:
some_y = A(some_x)
some_y.plot()

##### <font color=#8B4513> Exercise: </font>
  * Is this true for you: **"I can, mathematically, write a definition of a 1D Poisson forward model that maps the conductivity field to the temperature distribution field"**? If not, what questions do you have?
  * Trying a different conductivity profile: 
      * Create another constant conductivity profile `another_x` of value 20 as a `CUQIarray` object.
      * Evaluate the forward model at the new conductivity profile and store the result in a variable `another_y`. Plot the solution.
      * In one plot, compare the solutions `some_y` and `another_y` using the `plot` method of the `CUQIarray` object. What do you observe? and why?
  * Experimenting with setting up the `map` in `Poisson1D`:
      * Execute `help(Poisson1D)`
      * Note the `map` parameter of `Poisson1D`. We can use this parameter to transform the conductivity field before solving the PDE. Create the forward model again, name it `A_map`, with setting up `map` to be `lambda x: np.exp(x)` to ensure that the conductivity is always positive.
      * Inspect the domain and range geometries of the new forward model `A_map` what is different this time, and why?
      * Create a CUQIarray object `x_with_map` with a constant conductivity profile of value 0 and evaluate the forward model at `x_with_map` and store it in variable `y_with_map`. Make sure to use the right geometry for `x_with_map`.
      * Plot `x_with_map` using `x_with_map.plot()`, What do you observe?
      * Plot the solution `y_with_map` and `some_y` in the same plot. What do you observe? and why?
  * Experimenting with setting up the `field_type` in `Poisson1D`:
      * Note the `field_type` parameter of `Poisson1D`. We can use this parameter to change the parameterization of the conductivity field. Create the forward model again, name it `A_step`, with setting both the `map` as in the previous exercise and the `field_type` to be `"Step"` to generate a step parameterization of the conductivity field.
      * What is the domain and range geometries of the new forward model `A_step`?
      * Create `x_step` to be `x_step = CUQIarray(np.arange(A_step.domain_dim)*0.1, geometry=A_step.domain_geometry)`.
      * Plot `x_step` using `x_step.plot()`.
      * What does the dimension `A_step.domain_dim` represent? and is it equal to the size of the domain geometry grid?
      * Evaluate the forward model at `x_step` and store it in variable `y_step`. Plot the solution.
  * Is this true for you: **"I can load the pre-existing Poisson 1D forward model in CUQIpy and evaluate it on some input and visualize the input and output"**? If not, what questions do you have?

## Load modules
We start of by importing the Python packages we need (including CUQIpy).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import cuqi
from cuqi.testproblem import Deconvolution1D, Deconvolution2D
from cuqi.distribution import JointDistribution, Gaussian, CMRF, LMRF, GMRF
from cuqi.sampler import LinearRTO, pCN, CWMH, ULA, MALA, NUTS
from cuqi.problem import BayesianProblem


In [ ]:
list of samplers!!

# 1. Defining and sampling a Bayesian Inverse Problem (high-level)<a class="anchor" id="BIP"></a>

Solving a Bayesian inverse problem amounts to characterizing the posterior distribution.

The posterior describes the probability distribution of the parameters we are interested in. This is achieved by combining prior knowledge of the parameters and observed data. In its most general form, the posterior is given by Bayes' theorem:

\begin{align*}
p(\boldsymbol{\theta} \mid \mathbf{y}) = \frac{p(\mathbf{y} \mid \boldsymbol{\theta})p(\boldsymbol{\theta})}{p(\mathbf{y})} \propto p(\mathbf{y} \mid \boldsymbol{\theta})p(\boldsymbol{\theta}),
\end{align*}

where $\boldsymbol{\theta}$ is the parameter vector of *all* the parameters we are interested in inferring and $\mathbf{y}$ is the observable data. In the simplest case one could have a single parameter vector of interest, say $\boldsymbol{\theta}=[\mathbf{x}]$.

The probability density function $p(\boldsymbol{\theta})$ is the prior distribution of the parameters.

Given fixed observed data $\mathbf{y}_\mathrm{data}$, the term $p(\mathbf{y} \mid \boldsymbol{\theta})$ considered as a function of $\boldsymbol{\theta}$ is known as the
likelihood function or just *likelihood*, also denoted $L(\boldsymbol{\theta} \mid \mathbf{y} = \mathbf{y}_\mathrm{data})$, which we note is not a probability density but a density function.

When $\mathbf{y}$ is not fixed, $p(\mathbf{y} \mid \boldsymbol{\theta})$ is a probability density function of the data $\mathbf{y}$ given the value of the parameters $\boldsymbol{\theta}$. In CUQIpy we refer to this distribution as the *data distribution*.

The denominator $p(\mathbf{y})$ is the *evidence* and is a normalization constant (that we typically ignore because it does not affect the MCMC sampling) that ensures that the posterior integrates to 1.


### Note on Bayesian inverse problems with CUQIpy

CUQIpy uses a general approach to Bayesian modeling that not only aims to define the posterior distribution, but instead to define the joint distribution of all the parameters. This more general approach is useful because it allows one to carry out more tasks related to uncertainty quantification of inverse problems such as prior-predictive checks, model checking, posterior-predictive checks and more. For more details on some of these topics see the overview in [1].

In this notebook, we initially focus on how to define and sample a Bayesian inverse problem using the high-level interface in CUQIpy. We then later show a more low-level approach to defining the posterior distribution, which is useful for users who want to have more control the type of sampler used for sampling the posterior distribution.

## 1.1 Deterministic forward model and observed data
Consider a Bayesian inverse problem
$$
\mathbf{y}=\mathbf{A}\mathbf{x} + \mathbf{e},
$$

where $\mathbf{A}: \mathbb{R}^n \to \mathbb{R}^m$ is the (deterministic) forward model of the inverse problem and $\mathbf{y}$ and $\mathbf{x}$ are random variables representing the observed data and parameter of interest respectively. Here $\mathbf{e}$ is a random variable representing additive noise in the data.

For this example let us consider the `Deconvolution1D` testproblem and extract a CUQIpy forward model and some synthetic data denoted $\mathbf{y}_\mathrm{data}$ (a realization of $\mathbf{y}$). 

Note that this is a linear inverse problem, but the same approach can be used for nonlinear inverse problems.

In [ ]:
# Load forward model, data and problem information
A, y_data, probInfo = Deconvolution1D(phantom="sinc").get_components()

# For convenience, we define the dimension of the domain of A
n = A.domain_dim

# For convenience, we extract the exact solution as x_exact
x_exact = probInfo.exactSolution


Before going further let us briefly visualize the data and compare with the exact solution to the problem.

Here we should expect to see that the data is a convolved version of the exact solution with some added noise. We can also inspect the `probInfo` variable to get further information about the problem.

In [ ]:
# Plot the data
plt.subplot(121); x_exact.plot(); plt.title('Exact Solution')
plt.subplot(122); y_data.plot(); plt.title('Data')

# Print information about the problem
print(probInfo)


## 1.2 Distributions for parameters and data

The goal now is to define distributions for each the random variables that represent the parameters and data in the above-mentioned Bayesian inverse problem. One way to think about this, is that we are trying to define a statistical model for the data generating process (see reference [1]), which we call a *Bayesian Problem* in CUQIpy.

For the unknown $\mathbf{x}$, we use a-priori knowledge to define its distribution. In this case, the sinc phantom in the Deconvolution test problem can be represented fairly well by a distribution that generates smooth, but spatially correlated realizations.

One such distribution is the Gaussian Markov Random Field (GMRF) distribution. This distribution assumes a Gaussian distribution on the differences between neighboring elements of $\mathbf{x}$, i.e. in 1D:

\begin{align*}
x_i - x_{i-1} \sim \mathrm{Gaussian}(0, d^{-1}), \quad i=1, \ldots, n,
\end{align*}

where we purposely leave out the details on the boundary conditions for this notebook.

To simplify the notation, we denote by *GMRF* the distribution that induces this property on a vector $\mathbf{x}$ defined by its mean and precision $d$. That is, the above can be written as

\begin{align*}
\mathbf{x} &\sim \mathrm{GMRF}(\mathbf{0}, d),
\end{align*}


with some choice of the precision say $d=50$. For more details on GMRF see the first CUQIpy paper [2].

The GMRF distribution is implemented in CUQIpy as [GMRF class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.GMRF.html#cuqi.distribution.GMRF) and can be used as follows:

In [ ]:
# Define prior precision
d = 50

# Define GMRF prior (zero boundary conditions are the default)
x = GMRF(np.zeros(n), d)


From the problem info string earlier, we saw that the noise is Gaussian with standard deviation $s_\mathbf{e}=0.01$, i.e.,

\begin{align*}
\mathbf{e} &\sim \mathrm{Gaussian}(\mathbf{0}, s_\mathbf{e}^2 \mathbf{I}).
\end{align*}

Instead of working directly with the noise variable $\mathbf{e}$, we can include the stochastic element of the noise in the random variable $\mathbf{y}$ representing the data.

Because the data depend on $\mathbf{x}$ we are really interested in defining the *data distribution* for $\mathbf{y} \mid \mathbf{x}$ and since the noise is the only stochastic element of $\mathbf{y}$ when $\mathbf{x}$ is fixed we have

\begin{align*}
\mathbf{y} \mid \mathbf{x} &\sim \mathrm{Gaussian}(\mathbf{A}\mathbf{x}, s_\mathbf{y}^2 \mathbf{I}),
\end{align*}

where $s_\mathbf{e}=s_\mathbf{y} = 0.01$.

Notice that this definition depends both on the forward model $\mathbf{A}$ and the random variable $\mathbf{x}$.

In CUQIpy, we can define the distribution for $\mathbf{y} \mid \mathbf{x}$ matching the mathematical expression using our previously defined variables `A` and `x` as follows:

In [ ]:
# Define noise standard deviation
s_y = 0.01

# Define data distribution
y = Gaussian(A@x, s_y**2)


#### ★ Try yourself (optional):  

Have a look at the distributions for $\mathbf{x}$ and $\mathbf{y}$ by calling `print` on them in the code-cell below. 
- How are the distributions different?
- Is it clear that the distribution for $\mathbf{y}$ is a conditional distribution?
- Can you generate and plot a realization (sample) of $\mathbf{x}$? Does the realization show spatial correlation?
- How about generating a sample from $\mathbf{y}\mid\mathbf{x}$ given some fixed value of $\mathbf{x}$? What does a sample represent in this case?

**Hint:** to condition `y` on a value e.g. `x_exact` the syntax is `y(x=x_exact)`

In [ ]:
# Your code here




### Note on notation

It is common (for convenience in terms of notation) not to explicitly write the dependance of each random variable when specifying a complete Bayesian problem. For example, for the case above one would often write
\begin{align*}
\mathbf{x} &\sim \mathrm{GMRF}(\mathbf{0}, d)\\
\mathbf{y} &\sim \mathrm{Gaussian}(\mathbf{A}\mathbf{x}, s_\mathbf{y}^2 I),
\end{align*}

where the dependance of $\mathbf{y}$ on $\mathbf{x}$ is implicit.

This compact notation completely specifies the Bayesian problem for the so-called *data generating process*, making clear all the assumptions about the data and parameters.

In CUQIpy - when all deterministic parameters and forward models are defined - the Bayesian problem is written in code using almost exactly the same syntax as the mathematical notation above.

In [ ]:
# Bayesian problem (repeated in case previous cells were modified)
x = GMRF(np.zeros(n), 50)
y = Gaussian(A@x, s_y**2)


# 1.3 Bayesian inverse problem
The simplest way to sample a Bayesian inverse problem in CUQIpy is to use the [BayesianProblem class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.problem/cuqi.problem.BayesianProblem.html#cuqi.problem.BayesianProblem).

Using the BayesianProblem class, one can easily define and sample from the posterior distribution of a Bayesian inverse problem by providing the distributions for the parameters and data and subsequently setting the observed data.

Calling the `UQ` method on the BayesianProblem object will automatically sample the posterior distribution using a suitable choice of sampling algorithm and plot the results.

In [ ]:
# The simplest way to do uncertainty quantification in CUQIpy:

BP = BayesianProblem(x, y)      # Create Bayesian problem
BP.set_data(y=y_data)           # Provide observed data
samples = BP.UQ(exact=x_exact)  # Run UQ analysis (comparing with exact solution)


In the above example, we provided our assumptions about the data generating process by defining the distributions for the parameters and data and provided the observed data for the problem. CUQIpy then automatically sampled the posterior distribution and a credibility interval for the parameter $\mathbf{x}$ as well as the mean of the posterior was plotted and compared to the ground truth (x_exact).

We see that compared to the ground truth, the mean of the posterior is a good estimate and the credibility interval contains the ground truth.

In the next section, we show how to define the posterior distribution more explicitly and how to sample it with a specific choice of sampler.

# 2 Defining and sampling a Bayesian Inverse Problem (low-level)<a class="anchor" id="Joint"></a>

Instead of relying on the BayesianProblem class to automatically sample the posterior distribution (which does not always work well), we can also define the posterior distribution explicitly and sample it using a specific sampler in CUQIpy. This requires a bit more work, but it gives us more control over the sampling process.

## 2.1 Defining the joint distribution
In the Bayesian framework, we are interested in the joint distribution over $\mathbf{x}$ and $\mathbf{y}$. The joint distribution is defined as the product of the individual probability density functions. In our simple case, the joint distribution can be described in density form as

$$
p(\mathbf{y},\mathbf{x}) = p(\mathbf{y} \mid \mathbf{x})p(\mathbf{x}).
$$

In CUQIpy, if we want more control, we can as an alternative to using BayesianProblem, instead define the joint distribution ourselves by passing each of the distributions as arguments to the [JointDistribution class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.JointDistribution.html#cuqi.distribution.JointDistribution).

In [ ]:
# Define joint distribution p(y,x)
joint = JointDistribution(y, x)


Calling `print` on the joint distribution gives a nice overview matching the mathematical description above.

In [ ]:
print(joint)


For the purpose of this notebook we are not going to dive further into the details of the joint distribution, and simply use it to define the posterior.

## 2.2 Defining the posterior distribution

To define the posterior distribution, we have to condition the joint distribution $p(\mathbf{y},\mathbf{x})$ on the observed data, say $\mathbf{y}_\mathrm{data}$. This is done by using Bayes' theorem, which in our simple case can easily be derived as

\begin{align*}
p(\mathbf{x} \mid \mathbf{y}=\mathbf{y}_\mathrm{data}) \propto L(\mathbf{x} \mid \mathbf{y}=\mathbf{y}_\mathrm{data})p(\mathbf{x}),
\end{align*}

where we use the notation $L(\mathbf{x} \mid \mathbf{y}=\mathbf{y}_\mathrm{data}) := p(\mathbf{y}=\mathbf{y}_\mathrm{data} \mid \mathbf{x})$ for the likelihood function to emphasize that, in the context of the posterior where $\mathbf{y}$ is fixed to $\mathbf{y}_\mathrm{data}$, it is a function of $\mathbf{x}$ and not on $\mathbf{y}$. In CUQIpy we sometimes use the short-hand printing style `L(x|y)` for brevity.

CUQIpy can automatically derive the posterior distribution for any joint distribution by passing the observed data as an argument to the "call" (condition) method of the joint distribution.  This is done as follows:

In [ ]:
posterior = joint(y=y_data) # Condition p(x,y) on y=y_data. Applies Bayes' rule automatically


We can now inspect the posterior distribution by calling `print` on it. Notice that the posterior equation matches the mathematical expression above.

In [ ]:
print(posterior)


#### ★ Try yourself (optional):  
The posterior is essentially just another CUQIpy distribution. Have a look at the [Posterior class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.Posterior.html) in the online documentation to see what attributes and methods are available.

Try evaluating the posterior log probability density function (logpdf) at some point say $\mathbf{1}$ and $2\cdot\mathbf{1}$, where $\mathbf{1}$ is a ones-vector.

Try evaluating the posterior pdf at the same points. Can you explain why the pdf gives the same result for both points?

In [ ]:
# Your code here




# 2.3 Sampling the posterior

Now that we have defined the posterior distribution for our parameter of interest $\mathbf{x}$ given $\mathbf{y}_\mathrm{data}$, we can characterize the parameter and its uncertainty by samples from the posterior distribution. However, in general the posterior is not a simple distribution that we can easily sample from. Instead, we need to rely on Markov Chain Monte Carlo (MCMC) methods to sample from the posterior.

In CUQIpy, a number of MCMC samplers are provided in the sampler module that can be used to sample probability distributions. All samplers have the same signature, namely `Sampler(target, ...)`, where `target` is the target CUQIpy distribution and `...` indicates any (optional) arguments.

In the case of the posterior above, which is defined from a linear model and Gaussian likelihood and prior, the Linear Randomize-then-Optimize [(LinearRTO)](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.LinearRTO.html#cuqi.sampler.LinearRTO) sampler is a good choice to efficiently generate samples. This is also the sampler chosen by the BayesianProblem class for this problem.

 Like any of the other samplers, we set up the sampler by simply providing the target distribution.

In [ ]:
sampler = LinearRTO(posterior)


After the sampler is defined we can compute samples via the `sample` method.

In [ ]:
samples = sampler.sample(500)


Similar to directly sampling distributions in CUQIpy, the returned object is a `cuqi.samples.Samples` object.

This object has a number of methods available. In this case, we are interested in evaluating if the sampling went well. To do this we can have a look at the chain history for 2 different parameters.

In [ ]:
samples.plot_chain([30, 45]);


In both cases the chains look very good with no discernible difference between the start and end of the chain. This is a good indication that the chain has converged and there is little need for removing samples that are part of a "burn-in" period. In practice, the samples should be inspected with more rigor to ensure that the MCMC chain has converged, but this is outside the scope of this notebook.

The good sampling is in large part due to the LinearRTO sampler, which is built specifically for the type of problem of this example. For the sake of presentation let us remove the first 100 samples using the `burnthin` method (see [samples.burnthin](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.samples/cuqi.samples.Samples.burnthin.html#cuqi.samples.Samples.burnthin)) and store the "burnthinned" samples in a new variable.

In [ ]:
samples_final = samples.burnthin(Nb=100)


Finally, we can plot a credibility interval of the samples and compare to the exact solution (from `probInfo`).

This is what the `UQ` method of the BayesianProblem class did under the hood for us earlier.

In [ ]:
samples_final.plot_ci(95, exact=probInfo.exactSolution)


### Trying out other samples

The LinearRTO sampler can only sample Gaussian posteriors that also have an underlying linear model.

It is possible to try out other CUQIpy samplers (which also work for a broader range of problems). For example:

* **[pCN](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.pCN.html#cuqi.sampler.pCN)** - preconditioned Crank-Nicolson sampler.
* **[CWMH](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.CWMH.html)** - Component-wise Metropolis-Hastings sampler.
* **[ULA](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.ULA.html)** - Unadjusted Langevin Algorithm.
* **[MALA](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.MALA.html)** - Metropolis Adjusted Langevin Algorithm.
* **[NUTS](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.NUTS.html)** - No U-Turn Sampler: A variant of the Hamiltonian Monte Carlo sampler well-established in literature.

Note in particular that ULA, MALA and NUTS all require the gradient of the logpdf. This is handled automatically in CUQIpy for linear models.

#### ★ Try yourself (optional):  

Try sampling the posterior above using one of the suggested samplers (click the links to look at the online documentation for the sampler to get more info on it).

Compare results (chain, credibility interval etc.) to the results from LinearRTO.

All the suggested samplers (except NUTS) will likely require > 5000 samples to give reasonable results, and for some playing with step sizes (scale) is needed. This is because they are not as efficient as LinearRTO or NUTS. For some samplers, the method [sample_adapt](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.NUTS.sample_adapt.html#cuqi.sampler.NUTS.sample_adapt) will auto-scale the step size according to some criteria, e.g. reach approximately optimal acceptance rate and a burn-in should be added to specify how many samples to use for the adaptation.



In [ ]:
# Your code here





# 3. Exploring different prior choices<a class="anchor" id="ModifyPriors"></a>

In the above example, we used a GMRF prior for the parameter $\mathbf{x}$. However, it is not always obvious what prior to use for a given problem. In such cases, it is often useful to try out different priors to see how they affect the posterior distribution.

In CUQIpy, it is easy to modify the prior and re-sample the posterior distribution. This is most easily done by using the BayesianProblem class.

####  Try yourself (optional):  

Please carry out the following exercise to see how the prior affects the posterior distribution. 

Note that: here we use the [sample_posterior](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.problem/cuqi.problem.BayesianProblem.sample_posterior.html#cuqi.problem.BayesianProblem.sample_posterior) method of the BayesianProblem class to sample the posterior distribution and store the samples without plotting. We then manually plot the samples using the `plot_ci` method of the `Samples` object.

- Try another prior such as [LMRF](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.LMRF.html#cuqi.distribution.LMRF) or [CMRF](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.CMRF.html#cuqi.distribution.CMRF) for the 1D case (look up appropriate arguments in the documentation) using [`BayesianProblem`](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.problem/cuqi.problem.BayesianProblem.html#cuqi.problem.BayesianProblem).
- Try switching the testproblem from Deconvolution1D to [Deconvolution2D](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Deconvolution2D.html#cuqi.testproblem.Deconvolution2D) (look up appropriate arguments in the documentation).
- You may also try defining your own Bayesian inverse problem using this interface. ★

In [ ]:
# Your code here
# You can modify this code or write your own from scratch

# 1. Forward model and data
A, y_data, probInfo = Deconvolution1D(phantom="sinc").get_components()

# 2. Distributions
x = GMRF(np.zeros(A.domain_dim), 50) # Try e.g. LMRF or CMRF (also update scale parameters!)
y = Gaussian(A@x, 0.01**2)

# 3. Bayesian problem
BP = BayesianProblem(y, x).set_data(y=y_data)

# 4. Sample posterior
samples = BP.sample_posterior(500)

# 5. Analyze posterior
samples.plot_ci(exact=probInfo.exactSolution)


You may have noticed that finding suitable parameters for the prior could be a challenge. To see how to automatically find suitable parameters for the prior, see the [Gibbs notebook](Exercise06_Gibbs.ipynb).

# 4. Computing point estimates of the posterior ★ <a class="anchor" id="pointestimates"></a>

In addition to sampling the posterior, we can also compute point estimates of the posterior. A common point estimate to consider is the Maximum A Posteriori (MAP) estimate, which is the value of the Bayesian parameter that maximizes the posterior density. That is,

\begin{align*}
\mathbf{x}_\mathrm{MAP} = \arg\max_\mathbf{x} p(\mathbf{x} \mid \mathbf{y}_\mathrm{data}).
\end{align*}

The easiest way to compute the MAP estimate is to use the `MAP` method of the `BayesianProblem` class as follows:

In [ ]:
# We redefine in case something was changed

# Deterministic forward model
A, y_data, probInfo = Deconvolution1D(phantom="sinc").get_components()

# Distributions for each parameter
x = GMRF(np.zeros(A.domain_dim), 50)
y = Gaussian(mean=A@x, cov=0.01**2)

# Define Bayesian problem
BP = BayesianProblem(y, x).set_data(y=y_data)


In [ ]:
x_map = BP.MAP() # Maximum a posteriori estimate


The automatic solver selection is also still work-in-progress.

After we have computed the MAP, we can then estimate to the exact solution (from `probInfo`)

In [ ]:
x_map.plot()
plt.title('MAP estimate')
plt.show()

probInfo.exactSolution.plot()
plt.title('Exact solution')
plt.show()


#### ★ Try yourself (optional):  

- Try switching to the Deconvolution2D testproblem. You may have to play with the prior standard deviation to get a good MAP estimate.
- Try switching the prior to a CMRF distribution for the 1D case. Does the MAP estimate change?

## Story 1: MH adaptive scaling, donuts (it pays off to adapt)

## Sample a target

In [ ]:
from cuqi.distribution import DistributionGallery
import numpy as np
import matplotlib.pyplot as plt
from cuqi.sampler import MH, CWMH
import time

target = DistributionGallery("donut")


print(target)






# MH
# try scale = 0.1, 1, 10
Ns = 30000
Nb = 5000
scale = 0.05
MH_sampler = MH(target, scale=scale, x0=np.array([0,0]))
ti = time.time()
MH_fixed_samples = MH_sampler.sample(Ns, Nb)
print('Elapsed time MH:', time.time() - ti, '\n')
MH_fixed_samples.plot_pair(ax=plt.gca())


# try sample adapt
MH_adapted_samples = MH_sampler.sample_adapt(Ns, Nb)
print('Elapsed time MH:', time.time() - ti, '\n')

#set defult color matplotlib
MH_adapted_samples.plot_pair(ax=plt.gca(), scatter_kwargs={'c':'r'})


#CWMH_sampler = CWMH(target, scale=scale, x0=np.array([0,0]))
#CWMH_fixed_samples = CWMH_sampler.sample(Ns, Nb)
#print('Elapsed time CWMH:', time.time() - ti, '\n')
#CWMH_fixed_samples.plot_pair(ax=plt.gca(), scatter_kwargs={'c':'g'})
#
#
#CWMH_adapted_samples = CWMH_sampler.sample_adapt(Ns, Nb)
#print('Elapsed time CWMH:', time.time() - ti, '\n')
#CWMH_adapted_samples.plot_pair(ax=plt.gca(), scatter_kwargs={'c':'y'})



what do you notice about acceptance rate?
scale?
plot diagnostics, ESS

try 60000 / 10000


In [ ]:
print(MH_fixed_samples.compute_ess())
print(MH_adapted_samples.compute_ess())
MH_fixed_samples.plot_trace()
MH_adapted_samples.plot_trace()


## story 2: Poisson, CW vs MH  (not all unkowns are created equal)

In [ ]:
# Poisson inverse problem
from cuqi.testproblem import Poisson1D
from cuqi.distribution import GMRF, Gaussian, CMRF
from cuqi.problem import BayesianProblem
from cuqi.sampler import MH, CWMH, pCN

# Load forward model, data and problem information
n = 20
Ns = 3000
Nb = 2000
cov = 4.62905925e-05
A, y_data, probInfo = Poisson1D(dim=n).get_components()

x = GMRF(np.zeros(A.domain_dim), 8, geometry=A.domain_geometry,  order=1)
y = Gaussian(A(x), cov)

BP = BayesianProblem(x, y).set_data(y=y_data)
posterior = BP.posterior()

scale = 0.06
MH_sampler = MH(posterior, scale = scale, x0=np.ones(n))
MH_samples = MH_sampler.sample_adapt(Ns, Nb)


CWMH_sampler = CWMH(posterior, scale = scale, x0=np.ones(n))
CWMH_samples = CWMH_sampler.sample_adapt(Ns, Nb)
int(0.1*Ns)

In [ ]:
MH_samples.burnthin(int(0.1*(Ns))).plot_ci(95, exact=probInfo.exactSolution)
plt.figure()
CWMH_samples.burnthin(int(0.1*Ns)).plot_ci(95, exact=probInfo.exactSolution)



In [ ]:
CWMH_sampler.scale


time both
insptect scale for both
use KL expansion and compare both samplers 

In [ ]:
print(MH_samples.compute_ess())
print(CWMH_samples.compute_ess())

In [ ]:
test = Poisson1D(dim=n)

In [ ]:
test.likelihood.distribution.cov

## Story 3: Gradient info, ULA (univariate, bias) (Wandering randomly or be Guided by the force / let the force guide you)

## Story 4: Gradient info, MALA (is it ever possible to fix the bias)

## Story 5: NUTS (it is time to go nuts with nuts)

inspect sampler / its functions